In [1]:
import pandas as pd
from tensorflow import keras
from matplotlib import pyplot as plt
# from sklearn.preprocessing import StandardScaler
import sklearn
import numpy as np
import pandas as pd
import seaborn as sns
from datetime import datetime
import os
import statistics as sts
from scipy.stats import norm
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

2025-09-23 00:43:32.981508: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1758613413.000267    8751 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1758613413.005793    8751 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1758613413.020630    8751 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1758613413.020671    8751 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1758613413.020674    8751 computation_placer.cc:177] computation placer alr

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

In [ ]:
data = pd.read_csv('databases/all_profiles2.csv')


# data.index = pd.to_datetime(data['Date Time'], format='%d.%m.%Y %H:%M:%S')
# data['Date Time'] = pd.to_datetime(data['Date Time'], format='%d.%m.%Y %H:%M:%S')
# data['Date Time'] = pd.to_datetime(data['Time'], unit='ms')
# data = data[::10]

data = data.dropna()
data